## Get better at dask dataframes

In this lesson you will learn some good practices for dask dataframes and dealing with data in general.


### Work close to your data

To get started when you are working with data that is in the cloud it's always better to work close to your data, to minimize the impact of IO networking. 

In this lesson, we will use coiled clusters that will be created on the same region that our datasets are stored. (the region is `"us-east-2"`)

**NOTE:**
If you do not have access to a coiled cluster you, can follow along just make sure you use the smaller dataset (use the `"0.5GB-"` ones). 

## Parquet vs CSV

Most people are familiarized with csv files, but when it comes to working with data, working with parquet can make a big difference. The Parquet file format is column-oriented and it's designed to efficiently store and retrieve data. 

**Extra reading**
You can read of the multiple advantages of using parquet data format in the blog [Advantages of Parquet File Format](https://www.coiled.io/blog/parquet-file-column-pruning-predicate-pushdown).

Let's see an example where we compare reading the same data but in one case it is stored as `csv` files, while the other as `parquet` files. 





###NOTES ON CSV AND PARQUET

CSV: ROW BASED 

PARQUET: COLUMN BASED

put image

- pq suppots predicate pushdown, csv does not: apply filter first then select data
- pq stores schema information, 
- parquet has a metadata that makes it really efficient (row groups )

In [1]:
data ={"0.5GB-csv": "s3://coiled-datasets/h2o-benchmark/N_1e7_K_1e2/*.csv",
       "0.5GB-pq": "s3://coiled-datasets/h2o-benchmark/N_1e7_K_1e2_parquet/*.parquet",
       "5GB-csv": "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2/*.csv",
       "5GB-pq": "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_parquet/*.parquet",}

In [2]:
import coiled
from dask.distributed import Client
import dask.dataframe as dd

### SECTION ON HOW TO LOGIN INTO COILED WHEN WE HAVE INFO

In [ ]:
#cluster = coiled.Cluster(name="dask-tutorial")

In [3]:
%%time
cluster = coiled.Cluster(name="dask-tutorial",
                        n_workers=8,
                        package_sync=True,
                        backend_options={"region_name": "us-east-2"},
                        );

## maybe use mi6 instead, the default ones are slower...

╭─────────────────────────────────────── Package Sync Notes ───────────────────────────────────────╮
│                                      ╷                                                           │
│   Package                            │ Note                                                      │
│ ╶────────────────────────────────────┼─────────────────────────────────────────────────────────╴ │
│   coiled-runtime                     │ Package ignored, no risk                                  │
│   icu                                │ Package ignored, no risk                                  │
│   libabseil                          │ Package ignored, no risk                                  │
│   openssl                            │ Package ignored, no risk                                  │
│                                      ╵                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

{'channel': 'conda-forge', 'sdist': None, 'source': 'conda', 'conda_name': 'zstd', 'name': 'zstd', 'client_version': '1.5.2', 'specifier': '', 'include': False, 'note': None, 'error': '1.5.2 has no install candidate for linux-64', 'md5': None} -1
{'channel': 'conda-forge', 'sdist': None, 'source': 'conda', 'conda_name': 'zlib', 'name': 'zlib', 'client_version': '1.2.13', 'specifier': '', 'include': False, 'note': None, 'error': '1.2.13 has no install candidate for linux-64', 'md5': None} -1
{'channel': 'conda-forge', 'sdist': None, 'source': 'conda', 'conda_name': 'zeromq', 'name': 'zeromq', 'client_version': '4.3.4', 'specifier': '', 'include': False, 'note': None, 'error': '4.3.4 has no install candidate for linux-64', 'md5': None} -1
{'channel': 'conda-forge', 'sdist': None, 'source': 'conda', 'conda_name': 'xz', 'name': 'xz', 'client_version': '5.2.6', 'specifier': '', 'include': False, 'note': None, 'error': '5.2.6 has no install candidate for linux-64', 'md5': None} -1
{'channel'

╭───────────────────────────────────────── Package Issues ─────────────────────────────────────────╮
│                    ╷                                                                   ╷         │
│   Package          │ Error                                                             │ Risk    │
│ ╶──────────────────┼───────────────────────────────────────────────────────────────────┼───────╴ │
│   zstd             │ 1.5.2 has no install candidate for linux-64                       │         │
│   zlib             │ 1.2.13 has no install candidate for linux-64                      │         │
│   zeromq           │ 4.3.4 has no install candidate for linux-64                       │         │
│   xz               │ 5.2.6 has no install candidate for linux-64                       │         │
│   tk               │ 8.6.12 has no install candidate for linux-64                      │         │
│   snappy           │ 1.1.9 has no install candidate for linux-64                       │         │
│   readline         │ 8.1.2 has no install candidate for linux-64                       │         │
│   re2              │ 2022.06.01 has no install candidate for linux-64                  │         │
│   orc              │ 1.8.0 has no install candidate for linux-64                       │         │
│   openjpeg         │ 2.5.0 has no install candidate for linux-64                       │         │
│   libxml2          │ 2.10.3 has no install candidate for linux-64                      │         │
│   libtiff          │ 4.4.0 has no install candidate for linux-64                       │         │
│   libthrift        │ 0.16.0 has no install candidate for linux-64                      │         │
│   libssh2          │ 1.10.0 has no install candidate for linux-64                      │         │
│   libsqlite        │ 3.40.0 has no install candidate for linux-64                      │         │
│   libprotobuf      │ 3.21.10 has no install candidate for linux-64                     │         │
│   libpng           │ 1.6.39 has no install candidate for linux-64                      │         │
│   liblapack        │ 3.9.0 has no install candidate for linux-64                       │         │
│   libgrpc          │ 1.49.1 has no install candidate for linux-64                      │         │
│   libgfortran      │ 5.0.0 has no install candidate for linux-64                       │         │
│   libffi           │ 3.4.2 has no install candidate for linux-64                       │         │
│   libevent         │ 2.1.10 has no install candidate for linux-64                      │         │
│   libcurl          │ 7.86.0 has no install candidate for linux-64                      │         │
│   libcrc32c        │ 1.1.2 has no install candidate for linux-64                       │         │
│   libcblas         │ 3.9.0 has no install candidate for linux-64                       │         │
│   libblas          │ 3.9.0 has no install candidate for linux-64                       │         │
│   lerc             │ 4.0.0 has no install candidate for linux-64                       │         │
│   krb5             │ 1.19.3 has no install candidate for linux-64                      │         │
│   grpcio           │ 1.49.1 has no install candidate for linux-64                      │         │
│   glog             │ 0.6.0 has no install candidate for linux-64                       │         │
│   gflags           │ 2.2.2 has no install candidate for linux-64                       │         │
│   freetype         │ 2.12.1 has no install candidate for linux-64                      │         │
│   bzip2            │ 1.0.8 has no install candidate for linux-64                       │         │
│   arrow-cpp        │ 10.0.1 has no install candidate for linux-64                      │         │
│                    ╵                                                                   ╵         │
╰────────────────────────────────────────────────────────────

Output()

CPU times: user 9.54 s, sys: 1.42 s, total: 11 s
Wall time: 1min 25s


In [4]:
client = Client(cluster)
client

<Client: 'tls://10.0.21.131:8786' processes=8 threads=32, memory=119.48 GiB>

In [11]:
ddf_csv = dd.read_csv(data["5GB-csv"], storage_options={"anon": True})
ddf_pq = dd.read_parquet(data["5GB-pq"], storage_options={"anon": True})
#dd.read_parquet(data["5GB-pq"], storage_options={"anon": True})

In [12]:
ddf_csv

,id1,id2,id3,id4,id5,id6,v1,v2,v3
npartitions=100,,,,,,,,,
,object,object,object,int64,int64,int64,int64,int64,float64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [13]:
ddf_pq

,id1,id2,id3,id4,id5,id6,v1,v2,v3
npartitions=100,,,,,,,,,
,category[unknown],category[unknown],category[unknown],Int32,Int32,Int32,Int32,Int32,float64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [10]:
ddf_csv

,id1,id2,id3,id4,id5,id6,v1,v2,v3
npartitions=100,,,,,,,,,
,category[unknown],category[unknown],category[unknown],Int32,Int32,Int32,Int32,Int32,float64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [9]:
%%time
ddf_csv.groupby("id1").agg({"v1": "sum"}).compute()

CPU times: user 315 ms, sys: 105 ms, total: 420 ms
Wall time: 2min 25s


,v1
id1,
id048,2000740
id080,2001520
id035,1999546
id086,2001468
id009,1997231
...,...
id087,2000103
id037,1996179
id008,1999858


In [7]:
%%time
ddf_pq.groupby("id1").agg({"v1": "sum"}).compute()

CPU times: user 40.4 ms, sys: 24.3 ms, total: 64.7 ms
Wall time: 3.07 s


,v1
id1,
id048,2000740
id080,2001520
id035,1999546
id086,2001468
id009,1997231
...,...
id087,2000103
id037,1996179
id008,1999858


Notice that the `parquet` version without doing much it is already ~5X faster. 

Let's take a look at the dtypes in both cases and see if we can make some things faster:

In [ ]:
ddf_csv

In [8]:
#IF I SPECIFY THE DTYPES THIS GETS MUCH SLOWER ??? Thoughts??

ddf_csv = dd.read_csv(
            data["5GB-csv"],
            dtype={
                "id1": "category",
                "id2": "category",
                "id3": "category",
                "id4": "Int32",
                "id5": "Int32",
                "id6": "Int32",
                "v1": "Int32",
                "v2": "Int32",
                "v3": "float64",
            },
            storage_options={"anon": True},)

In [ ]:
ddf_pq

In [ ]:
## example to exaplain column prunning. 


### Read about why in read_parquet we read the dtypes but not csv?

- show ddf.partitions[0].memory_usage(deep=True).compute() / 1e6
- see what happens with csv and with parquet, 

## dtypes

NOTE: 

FOR THE PUSPOSE OF THE TUTORIAL I NEED TO GENERATE THE DATA FOR 5GB WITH PYARROW STRINGS. 
OR TYPECAST, EXPLORE THAT.

THEN RUN 
```python
        ddf_q3 = ddf[["id3", "v1", "v3"]].astype({"id3": "string[pyarrow]"})
        (
            ddf_q3.groupby("id3", dropna=False, observed=True)
            .agg({"v1": "sum", "v3": "mean"})  
            .compute()
        )
```

chat with james to see if there is anything else about pyarrow dtypes we could be showing here.

## High cardinality 

- id1 has 100 unique values
- id3 has 1_000_000 unique values

Let's see what happens when we try to groupby on a high cardinality column, and what can we do to make this better. 

Read docs about shuffle, and explain advantages, extract useful info. Ask about p2p docs?
https://docs.dask.org/en/stable/dataframe-groupby.html#shuffle-methods


In [ ]:
#With 5 workers

In [ ]:
# CPU times: user 833 ms, sys: 338 ms, total: 1.17 s
# Wall time: 3min 9s

In [14]:
%%time
ddf = ddf_pq[["id3", "v1", "v3"]]
(
    ddf.groupby("id3")
    .agg({"v1": "sum", "v3": "mean"})
    .compute()
)

CPU times: user 644 ms, sys: 269 ms, total: 912 ms
Wall time: 2min 6s


,v1,v3
id3,,
id0000608844,156,45.221557
id0000466449,252,56.924430
id0000573987,151,50.175290
id0000776204,177,48.865545
id0000608718,190,52.758464
...,...,...
id0000821599,167,51.620558
id0000937302,197,45.701999
id0000248458,181,52.910110


2022-12-13 12:36:18,002 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-12-13 12:38:30,597 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/Users/ncclementi/mambaforge/envs/dask-tutorial/lib/python3.10/site-packages/distributed/comm/tcp.py", line 498, in connect
    stream = await self.client.connect(
  File "/Users/ncclementi/mambaforge/envs/dask-tutorial/lib/python3.10/site-packages/tornado/tcpclient.py", line 275, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ncclementi/mambaforge/envs/dask-tutorial/lib/python3.10/asyncio/tasks.py", line 456, in wait_for
    return fut.result()
asyncio.exceptions.CancelledError

The abo

In [ ]:
## Using shuffle tasks is slower :/ explanation?
##CPU times: user 1.58 s, sys: 858 ms, total: 2.44 s
#Wall time: 4min 49s

In [ ]:
%%time
ddf = ddf_pq[["id3", "v1", "v3"]]
(
    ddf.groupby("id3")
    .agg({"v1": "sum", "v3": "mean"}, shuffle="tasks")
    .compute()
)

In [ ]:
#THERE IS BUG, AND I CAN'T RUN THIS
# SEE https://github.com/dask/dask/issues/9754
%%time
ddf = ddf_pq[["id3", "v1", "v3"]]
(
    ddf.groupby("id3")
    .agg({"v1": "sum", "v3": "mean"}, shuffle="p2p")
    .compute()
)